In [1]:
!pip install tensorflow_text==2.15.0
#!pip install -q -U tensorflow-text==2.14.* tensorflow==2.14.*

!pip install evaluate
#!pip install bert_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency reso

In [2]:
import scipy
import tensorflow as tf
import tensorflow_text as text
import pickle

In [5]:
#reloaded = tf.saved_model.load('/content/drive/MyDrive/sememe_tranaslator_musa')
reloaded = tf.saved_model.load('fixed_48percent')

In [7]:
with open('decoded_sentences_awgn.pickle', 'rb') as handle:
    decoded_sentences_awgn = pickle.load(handle)
with open('decoded_sentences_rayleigh.pickle', 'rb') as handle:
    decoded_sentences_rayleigh = pickle.load(handle)

In [18]:
with open('test_raw.pickle', 'rb') as handle:
    test_raw = pickle.load(handle)

In [41]:
start_idx = 500
end_idx = 503

In [12]:
#!pip install nltk
translated_awgn = []
translated_rayleigh = []
for snr in range(7):
    tr_aw = []
    tr_ray = []
    for sent in decoded_sentences_awgn[snr]:
        tr_aw.append(reloaded(tf.constant(sent)).numpy().decode('utf-8'))
    for sent in decoded_sentences_rayleigh[snr]:
        tr_ray.append(reloaded(tf.constant(sent)).numpy().decode('utf-8'))
    translated_awgn.append(tr_aw)
    translated_rayleigh.append(tr_ray)




In [27]:
awgn_bleu_input = []
rayleigh_bleu_input = []
for snr in range(7):
  snr_ll_awgn = []
  for ll in translated_awgn[snr]:
      snr_ll_awgn.append(ll.split())
  awgn_bleu_input.append(snr_ll_awgn)
  snr_ll_rayleigh = []
  for ll in translated_rayleigh[snr]:
      snr_ll_rayleigh.append(ll.split())
  rayleigh_bleu_input.append(snr_ll_rayleigh)

In [31]:
raw_input = []
for sent in test_raw:
  raw_input.append(sent.split())

In [42]:
from nltk.translate import meteor
import nltk
nltk.download('wordnet')
awgn_snr = [-3,-2,-1,0,1,2,3]
rayleigh_snr  = [0,2,4,6,8,10,12]
def compute_meteor(ref_sentence_list, tx_sentence_list):
    N = len(tx_sentence_list)
    score = 0
    for s in range(N):
        score += meteor([ref_sentence_list[s]], tx_sentence_list[s])
    return score/N
print('AWGN Simulation METEOR Scores')
for i in range(7):

    K = compute_meteor(raw_input[start_idx:end_idx],awgn_bleu_input[i])
    print('SNR ' + str(awgn_snr[i]) + ': ', K)
print('Rayleigh Simulation METEOR Scores')
for s in range(7):
    K = compute_meteor(raw_input[start_idx:end_idx],rayleigh_bleu_input[s])
    print('SNR ' + str(rayleigh_snr[s]) + ': ', K)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import BertModel
#model  = SentenceTransformer("bertModel")
#model  = SentenceTransformer("sentence-transformers/all-miniLM")
model  = SentenceTransformer("bert-base-uncased")
#model = BertModel("bert-base-uncased")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [49]:
# see the details of model with print
embeddings_in = model.encode(test_raw[start_idx:end_idx])
print('AWGN BERT Scores')
for snr in range(7):
    embeddings_tx =  model.encode(translated_awgn[snr])
    similarities_sememe = cosine_similarity(embeddings_in,embeddings_tx)
    avg_sim = 0
    for i in range(len(similarities_sememe)):
        avg_sim += similarities_sememe[i,i]
    avg_sim = avg_sim / (i+1)
    print('SNR', awgn_snr[snr],': ', (avg_sim-0.4)/0.6)
print('Rayleigh BERT Scores')
for snr in range(7):
    embeddings_tx =  model.encode(translated_rayleigh[snr])
    similarities_sememe = cosine_similarity(embeddings_in,embeddings_tx)
    avg_sim = 0
    for i in range(len(similarities_sememe)):
        avg_sim += similarities_sememe[i,i]
    avg_sim = avg_sim / (i+1)
    print('SNR', rayleigh_snr[snr],': ', (avg_sim-0.4)/0.6)
